# dim_clientes

Esse notebook é responsável por criar uma tabela com as informações básicas dos clientes do e-commerce.


## Configurações Iniciais

### Importando as Bibliotecas

In [0]:
from pyspark.sql import functions as F

### Carregando os Dados

In [0]:
df_customers_dataset = (
    spark.read.csv(
        "/Volumes/workspace/default/e_commerce/olist_customers_dataset.csv",
        header=True,
        inferSchema=True
    )
    .drop("customer_city", "customer_state")
)

## Salvamento dos Dados

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ifrs_dev_ecommerce.gold.dim_clientes (
    customer_id STRING PRIMARY KEY COMMENT "ID do cliente",
    customer_zip_code_prefix INT COMMENT "Prefixo do CEP do cliente",
    client_name STRING COMMENT "Nome do cliente",
    CONSTRAINT fk_geolocalizacao_clientes FOREIGN KEY (customer_zip_code_prefix)
        REFERENCES ifrs_dev_ecommerce.gold.dim_geolocalizacao(geolocation_zip_code_prefix)
)
USING DELTA
CLUSTER BY (customer_id, customer_zip_code_prefix)
TBLPROPERTIES ("quality"="gold", "layer"="data-engineering")
COMMENT "Dimensão de clientes contendo informações de identificação e localização.";

-- Adicionando tags à tabela
ALTER TABLE ifrs_dev_ecommerce.gold.dim_clientes SET TAGS ("quality"="gold", "layer"="data-engineering");

In [0]:
(
    df_customers_dataset
    .write.mode("overwrite")
    .format("delta")
    .saveAsTable("ifrs_dev_ecommerce.gold.dim_clientes")
)